In [1]:
# Hepsi Burada case study <3
# You can find all information below about steps that I followed
# NLP methods that I worked with for this case is "TF IDF analysis with sklearn TfidfVectorizer func"
# I didnt calculate the importance of eventType column values because of my current job tasks.
# But I noted that what should I do with eventType column values in below.
# Also I scored tf idf results you can see below.
# And worked with 'title' column. I didnt use text column. Because I thought anyone who visiting a web site,
# Reads 'title' section first. Than clicks that article. So I decided that topic analysis should use 'title' section.
# I Worked with nltk library.
# References:
# https://analyticsindiamag.com/hands-on-implementation-of-tf-idf-from-scratch-in-python/
# https://www.freecodecamp.org/news/how-to-process-textual-data-using-tf-idf-in-python-cd2bbc0a94a3/
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
# https://www.holisticseo.digital/python-seo/tf-idf-analyse/
# https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
# https://www.datacamp.com/community/tutorials/text-analytics-beginners-nltk
# https://github.com/mayank408/TFIDF/blob/master/TFIDF.ipynb

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/volkan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Import article df
df_articles = pd.read_csv('articles.csv')

In [4]:
df_articles

,contentId,url,title,text,lang
0,-4110354420726924665,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,-7292285110016212249,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
2,-6151852268067518688,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
3,2448026894306402386,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
4,-2826566343807132236,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en
...,...,...,...,...,...
3042,9213260650272029784,https://startupi.com.br/2017/02/liga-ventures-...,"Conheça a Liga IoT, plataforma de inovação abe...","A Liga Ventures, aceleradora de startups espec...",pt
3043,-3295913657316686039,https://thenextweb.com/apps/2017/02/14/amazon-...,Amazon takes on Skype and GoToMeeting with its...,"Amazon has launched Chime, a video conferencin...",en
3044,3618271604906293310,https://code.org/about/2016,Code.org 2016 Annual Report,"February 9, 2017 - We begin each year with a l...",en
3045,6607431762270322325,https://www.bloomberg.com/news/articles/2017-0...,JPMorgan Software Does in Seconds What Took La...,"At JPMorgan Chase & Co., a learning machine is...",en


In [5]:
# Import interaction df
df_interactions = pd.read_csv('interactions.csv')

In [6]:
df_interactions

,personId,contentId,eventType
0,-8845298781299428018,-3499919498720038879,VIEW
1,-1032019229384696495,8890720798209849691,VIEW
2,-1130272294246983140,310515487419366995,VIEW
3,344280948527967603,310515487419366995,FOLLOW
4,-445337111692715325,-7820640624231356730,VIEW
...,...,...,...
72307,-9016528795238256703,-6590819806697898649,LIKE
72308,102305705598210278,-5813211845057621660,VIEW
72309,-9196668942822132778,-1999468346928419252,VIEW
72310,-9016528795238256703,-6590819806697898649,VIEW


In [7]:
# Preprocessing
# I checked how many records that each person had
df_interactions[['personId','contentId']].value_counts()

personId              contentId           
-709287718034731589   -330801551666885085     96
 2947963873628556360  -6642751159620064055    46
 983095443598229476   -133139342397538859     44
 8676130229735483748   3739926497176994524    44
 2195040187466632600  -1633984990770981161    40
                                              ..
 1539552466471655556  -4333957157636611418     1
                      -3248233692182971167     1
                      -1114438937697017987     1
                      -790959521412948853      1
-9223121837663643404  -8949113594875411859     1
Length: 40710, dtype: int64

In [8]:
# Merge 2 df's on 'contentId' column and assign to 'df'
df = pd.merge(df_articles, df_interactions, on=['contentId'])

In [9]:
# I checked if a person visited a site more than once. 
df[['contentId','url','title','personId','eventType']][df.duplicated()]

,contentId,url,title,personId,eventType
3,-6151852268067518688,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,-7267769888748948232,VIEW
22,3353902017498793780,https://www.cryptocoinsnews.com/ethereum-rise-...,The Rise And Growth of Ethereum Gets Mainstrea...,4340306774493623681,VIEW
23,3353902017498793780,https://www.cryptocoinsnews.com/ethereum-rise-...,The Rise And Growth of Ethereum Gets Mainstrea...,4340306774493623681,VIEW
26,3353902017498793780,https://www.cryptocoinsnews.com/ethereum-rise-...,The Rise And Growth of Ethereum Gets Mainstrea...,8676130229735483748,VIEW
27,3353902017498793780,https://www.cryptocoinsnews.com/ethereum-rise-...,The Rise And Growth of Ethereum Gets Mainstrea...,8676130229735483748,VIEW
...,...,...,...,...,...
72240,4675505028897335428,https://medium.mybridge.co/swift-top-10-articl...,Swift Top 10 Articles For The Past Year (v.2017),-1393866732742189886,VIEW
72241,4675505028897335428,https://medium.mybridge.co/swift-top-10-articl...,Swift Top 10 Articles For The Past Year (v.2017),-1393866732742189886,VIEW
72242,4675505028897335428,https://medium.mybridge.co/swift-top-10-articl...,Swift Top 10 Articles For The Past Year (v.2017),-1393866732742189886,VIEW
72247,4675505028897335428,https://medium.mybridge.co/swift-top-10-articl...,Swift Top 10 Articles For The Past Year (v.2017),-3643155458357242906,VIEW


In [10]:
# Check is there any missing values in df (If result = 'False', means that there is no missing values)
df.isnull().values.any()

False

In [11]:
# From here we can see that one personId has more than once visited the same article.
# So we can use TF/IDF method

In [12]:
# Checked the importance of eventType values
df.eventType.value_counts()

VIEW               61043
LIKE                5745
BOOKMARK            2463
COMMENT CREATED     1611
FOLLOW              1407
Name: eventType, dtype: int64

In [13]:
# VIEW -> 1
# LIKE -> 2
# FOLLOW -> 2
# COMMENT CREATED -> 3
# BOOKMARK -> 3
# In this section I thought that I can duplicate the data based on the values you can see above.
# But before using this method, I realised that I need to analyse much deeper than I could do.
# So I have yield about using this method with TF/IDF method. I accepted that there will be no difference
# Between "VIEW" and other values.

In [14]:
# Get unique person list
person_list = df.groupby(['personId']).size().index.to_list()
# For removing stop words section, get 'english' stopwords list
stop_words=stopwords.words('english')
# Create represantation df
dfRep = pd.DataFrame(person_list, columns = ['personId'])
# Create empty relevance column
dfRep['relevance'] = ''
# Start using TfidfVectorizer by every person in df. So start for loop with person_list
for person in person_list:
    # Create temp df for every person in for loop
    dfTemp = df[df.personId == person].copy()
    # Create empty dict for tf/idf output of every person
    textdict = {}
    # Create TfidfVectorizer func with stop words and using idf=True parameter.
    tfidf_vectorizer=TfidfVectorizer(stop_words='english',use_idf=True)
    # Vectorizing title column values 
    tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(dfTemp['title'])
    # Create a one vector for starting point of concanating df's section below 
    df_tfidf = pd.DataFrame(tfidf_vectorizer_vectors[0].T.todense(),index=tfidf_vectorizer.get_feature_names(),columns=["tfidf"])
    # For concanating vector df's create for loop in the range of vectors number
    for x in range(tfidf_vectorizer_vectors.shape[0]):
        df_tfidfTemp = pd.DataFrame(tfidf_vectorizer_vectors[x].T.todense(),index=tfidf_vectorizer.get_feature_names(),columns=["tfidf"])
        df_tfidf = pd.concat([df_tfidf,df_tfidfTemp])
    # After concanating all df's we can see that TF/IDF talks about docs one by one.
    # So I have use groupby.mean() method to calculate all TF/IDF values in one section.
    df_tfidf=df_tfidf.groupby(level=0).mean()
    # Sorting values by "tfidf" column value
    df_tfidf = df_tfidf.sort_values(by=["tfidf"],ascending=False)
    # Append df_tfidf values to represantation df 
    dfRep.loc[dfRep.personId == person,'relevance'] = df_tfidf.to_dict()

In [15]:
print(dfRep)

                 personId                                          relevance
0    -9223121837663643404  {'tfidf': {'android': 0.06640319788922187, 'go...
1    -9212075797126931087  {'tfidf': {'firebase': 0.21032169090943387, 'a...
2    -9207251133131336884  {'tfidf': {'aposta': 0.11547005383792518, 'na'...
3    -9199575329909162940  {'tfidf': {'advertising': 0.08545252741997275,...
4    -9196668942822132778  {'tfidf': {'docker': 0.19325702111033555, '15'...
...                   ...                                                ...
1890  9165571805999894845  {'tfidf': {'gartner': 0.2357022603955158, 'rep...
1891  9187866633451383747  {'tfidf': {'10': 0.0654728501098655, 'oferecen...
1892  9191849144618614467  {'tfidf': {'investir': 0.06666666666666667, 'a...
1893  9199170757466086545  {'tfidf': {'inovador': 0.0654728501098655, 'si...
1894  9210530975708218054  {'tfidf': {'devops': 0.08981056147499109, 'na'...

[1895 rows x 2 columns]
